![](../images/rivacon_frontmark_combined_header.png)

# Market Data Scenarios

In [ ]:
import seaborn as sn
import pyvacon.analytics as analytics
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
from pylab import rcParams
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import math
from numpy import arange
from numpy import meshgrid
import numpy as np
from ipywidgets import interact #, interactive, fixed, interact_manual, Layout
#import ipywidgets as widgets

import pyvacon.instruments.testdata as ins_testdata
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
import pyvacon.pricing.tools as pricing_tools
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline

refdate = analytics.ptime(2017,1,1,0,0,0) #dates which enters analytics objects must be analytics ptimes. 

## Setup market data and instrument

In [ ]:
# create sample data
#create sample dividend table

# dividend table neede fo forward curve
object_id = "TEST_DIV" 
ex_dates = converter.createPTimeList(refdate, [dt.datetime(2018,3,29), dt.datetime(2019,3,29), dt.datetime(2020,3,29), dt.datetime(2021,3,29)])
pay_dates = converter.createPTimeList(refdate, [dt.datetime(2018,4,1), dt.datetime(2019,4,1), dt.datetime(2020,4,1), dt.datetime(2021,4,1)])
tax_factors = analytics.vectorDouble([1.0, 1.0, 1.0, 1.0])
div_yield = analytics.vectorDouble([0, 0.005, 0.01, 0.01])
div_cash = analytics.vectorDouble([3.0, 2.0, 1.0, 0.0])
div_table=analytics.DividendTable(object_id, refdate, ex_dates, div_yield, div_cash, tax_factors, pay_dates)
spot = 100.0

# discount and borrow neede for forward curve
dates = converter.createPTimeList(refdate,[0, 365, 2*365, 3*365, 5*365])
df = analytics.vectorDouble([1.0,math.exp(-1.0*0.02), math.exp(-2.0*0.03), math.exp(-3.0*0.035), math.exp(-5.0*0.04)])
dc = analytics.DiscountCurve('TEST_DC', refdate, dates, df, enums.DayCounter.ACT365_FIXED, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)

df_bc = analytics.vectorDouble([1.0, math.exp(-1.0*0.05), math.exp(-2.0*0.05), math.exp(-3.0*0.05), math.exp(-5.0*0.05)])
bc = analytics.DiscountCurve('TEST_BC', refdate, dates, df_bc, enums.DayCounter.ACT365_FIXED, 
                             enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)
#forward curve
forward_curve = analytics.EquityForwardCurve('orig', refdate, spot, dc, bc, div_table)

### Volatility
#### SSVI
This parametrization is inspired by the volatility structure provided by stochastic volatility models. The total variance $w(k,t)$ for a strike log strike $k$ and time-to-maturity $t$ is given by
$$ w(k,t) = \frac{\theta_t}{2}\left( 1+\rho\phi(\theta_t)k+\sqrt{(\phi(\theta_t)k+\rho)^2+(1-\rho^2)}  \right) $$ 
and 
$$ \phi(\theta_t) = \frac{\eta}{\theta_t^\gamma(1+\theta_t)^{1-\gamma}} $$
for parameters $\rho$, $\eta$, $\gamma$ and given atm implied total variances $\theta_t:=\sigma^2(t)t$. The term structure of implied total variances is internally approximated by interpolation from given atm volatilities. The nice property of this surface is that there are very simple conditions on the parameters to guarantee that the surface is free of arbitrage, see [gatheral_jacquier_svi](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2033323).

In [ ]:
expiries = [1.0/12.0, 0.75, 1.0, 2.0]
atm_vols =  [0.3, 0.28, 0.25, 0.24]
gamma = 0.5
rho = -0.7
eta = 1.0
ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, atm_vols, rho, eta, gamma)
obj_id = 'TEST_SURFACE'
vol_surf = analytics.VolatilitySurface(obj_id, refdate, forward_curve, enums.DayCounter.ACT365_FIXED, ssvi_param)

### Down and Out Put

In [ ]:
#setup test instrument
rel_expiry = 365 # expiry in number of days
rel_level = 0.8 #barrier level relative to spot
rel_rebate = 0 # rebate relative to spot
rel_strike = 1.0 # strike relative to spot
dop = ins_testdata.DOP.__create_DOP__(spot, 'EUR', rel_expiry, rel_level, rel_strike, rel_rebate, 'TEST_UDL', refdate)

# setup pricing data
dop_pricing_data = analytics.LocalVolPdePricingData()
dop_pricing_data.pricer = 'LocalVolPdePricer'
dop_pricing_data.valDate = refdate
dop_pricing_data.pricingRequest = analytics.PricingRequest()
#dop_pricing_data.pricingRequest.s
dop_pricing_data.vol = vol_surf
dop_pricing_data.spec = dop
dop_pricing_data.param = analytics.PdePricingParameter()
dop_pricing_data.dsc = dc

## Market Data Scenarios

### Dividend Scenario

In [ ]:
# create scenario, set bucket and apply
#help(analytics.DividendScenario)
div_scenario = analytics.DividendScenario('DIV_SCEN', refDate = refdate, relativeDates=False, udls=[])
bucket_from = converter.getLTime(-50, ex_dates[1])
bucket_to = converter.getLTime(50, ex_dates[1])
#help(analytics.DividendScenario.setBucket)
div_scenario.setBucket(bucketFrom = bucket_from, bucketTo = bucket_to, relativeDates = False, 
                                     cashShiftAbs= 1.0, cashShiftRel = 1.10, yieldShiftAbs = 0, yieldShiftRel = 1.0, 
                                     taxShiftAbs = 0.0, taxShiftRel = 1.0)
#help(div_scenario.apply)
div_table_shifted = div_scenario.apply(div_table, refdate)
#help(analytics.EquityForwardCurve)
forward_curve_div_shifted = analytics.EquityForwardCurve('shifted div', refdate, spot, dc, bc, div_table_shifted)

In [ ]:
#plot the two forward curves
mkt_plot.curve(forward_curve, range(0,3*365), refdate)
mkt_plot.curve(forward_curve_div_shifted, range(0,3*365), refdate)


### Discount curve scenario

In [ ]:
# setup discoun curve scenario
bucket_from = converter.getLTime(-180, dates[1])
bucket_to = converter.getLTime(180, dates[1])
#hat scenario
dc_hat_scenario = analytics.DiscountCurveScenario('HAT_SCEN', refdate, False)
dc_hat_scenario.setHatBucketShift(fromBucket = bucket_from, midBucket = dates[1], toBucket = bucket_to, absShift = 0.005, relShift = 0.0)
dc_hat_shifted = dc_hat_scenario.apply(dc, refdate)
#constant scenario
dc_const_scenario = analytics.DiscountCurveScenario('CONST_SCEN', refdate, False)
dc_const_scenario.setConstantBucketShift(fromBucket = bucket_from, toBucket = bucket_to, absShift = 0.00, relShift = 0.1)
dc_const_shifted = dc_const_scenario.apply(dc, refdate)

In [ ]:
mkt_plot.curve(dc,range(0,3*365), refdate, True )
mkt_plot.curve(dc_hat_shifted,range(0,3*365), refdate, True)
mkt_plot.curve(dc_const_shifted,range(0,3*365), refdate, True)

### Volatility scenarios



### Bucket shifted volatility surface
The VolatilitySurfaceBucketShifted is the basis for the computation of vega buckets. This class provides th functionality to apply a bucket shift (bump of the volatility surface at a certain point). Up to now, two different bum-parametrizations are implemented:
- Radial basis function (RBF) with global support: For given strike-points $x_i$ and ttm $t_j$ one has the parametrization of the (i,j)-th bump by
$$b_{i,j}(x,t)=\frac{e^{(-s_x \cdot(x-x_i)^2 - s_t \cdot(t-t_j)^2)}}{\sum_i b_{i,j}(x,t)}$$ 
where the denominator ensures that we have a partition of unity 
- Transformed radial basis functions with local support: For given strike-points $x_i$ and ttm $t_j$ one has the parametrization of the i-th bump by:
$$b_{i,j}(x,t)=\frac{e^{-s_x / ( 1.0- ( 2.0(x-x_i)/(x_{i+1}-x_{i-1}) )^2 )}}{\sum_i b_{i,j}(x,t)}$$ 
for $x\in [x_{i-1},x_{i+1}]$ and $0$ otherwise

In [ ]:
#setup buckets and plot them
bucket_i = 1
bucket_j = 0
bucket_scale_x = 1000.0 #1000 #1.000 #
bucket_scale_t = 500.0 #500 #1.0000 #
shift = 0.001

bucket_strikes = arange(0.7, 1.4, 0.1) #[0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
bucket_expiries_integer = [30, 60, 180, 365, 365+180]
bucket_expiries = [x/365.0 for x in bucket_expiries_integer] #arange(30.0/365.0, 1.1, 30.0/365.0) 
#ttm = bucket_expiries_integer[bucket_j] # time to maturity in days
days_to_expiry = [bucket_expiries_integer[bucket_j-1], bucket_expiries_integer[bucket_j], bucket_expiries_integer[bucket_j+1]]

vol_bucket_surface = analytics.VolatilitySurfaceBucketShifted(vol_surf, bucket_strikes, bucket_expiries, shift)
    
def plot_surfaces(bucket_type):
    #create sample vol, apply bucket shift and plot
    
    # comment/uncomment to use the different bucket parametrizations
    if bucket_type == 'RBF':
        vol_bucket_surface.setRBF(bucket_scale_x, bucket_scale_t)
    else:
        vol_bucket_surface.setSimpleExponentialLocalSupport(bucket_scale_x, bucket_scale_t)
    vol_bucket_surface.setBucket(bucket_i, bucket_j)

    expiry = converter.getLTime(days_to_expiry[1], refdate)
    xstrikes = arange(0.4,1.8, 0.005)
    # plot vols at bucket expiry
    #
    #
    orix_x, orig_y = rcParams['figure.figsize'] 
    rcParams['figure.figsize'] = 18,10
    plt.subplot(2,3,1)

    mkt_plot.vol(vol_surf, days_to_expiry, xstrikes, refdate)
    plt.title('unshifted')
    plt.subplot(2,3,2)
    mkt_plot.vol(vol_bucket_surface, days_to_expiry, xstrikes, refdate)
    plt.title('bucket shifted')
    
    # calc difference
    
    diff = []
    for x in xstrikes:
        diff.append(vol_bucket_surface.calcImpliedVol(refdate, expiry, x)-vol_surf.calcImpliedVol(refdate, expiry, x))
    plt.subplot(2,3,3)
    plt.plot(xstrikes, diff, '-x')
    plt.title('diff')
    
    #plot color surf
    expiries = [x for x in range(30, 15*30, 10)]
    strikes =  arange(0.7, 1.4, 0.005)
    #plt.figure()
    #plt.pcolor(x,y,vega_bucket)
    plt.subplot(2,3,4)
    X,Y,vol_surf_un = mkt_plot.vol_surface(vol_surf, expiries, strikes, refdate )
    plt.subplot(2,3,5)
    X,Y,vol_surf_bucket = mkt_plot.vol_surface(vol_bucket_surface, expiries, strikes, refdate )
    plt.subplot(2,3,6)
    plt.pcolor(X,Y,vol_surf_bucket-vol_surf_un)
    plt.xlabel('x-strikes')
    plt.xlabel('ttm')
    rcParams['figure.figsize']  = orix_x, orig_y

interact(plot_surfaces, bucket_type = ['RBF', 'SIMPLE_EXP_LOCAL'])
#plt.pcolor(X,Y,vol_surf)

In [ ]:
#check for arbitrage
arbStrikes = analytics.vectorDouble()
arbExpiries = analytics.vectorDouble()
arbType = analytics.vectorString()
expiries = converter.createPTimeList(refdate, range(10,365, 5))
analytics.VolatilityCalibrator.checkArbitrage(arbStrikes, arbExpiries, arbType, vol_bucket_surface, 0.4, 1.6, 0.001, expiries, refdate, 0.000001)
mkt_plot.arbitrage_points(arbStrikes,arbExpiries,arbType, 0.4, 1.6, 3.0)
#vol =vol_surf.calcImpliedVol(refdate,converter.getLTime(refdate, 180), 1.0)

## Vega Buckets

In [ ]:
setting = {'RBF': (500.0, 250.0),
          'LOCAL_RBF': (10.0, 5.0)}
parametrization = 'RBF' #'RBF' # 'LOCAL_RBF'
vega_bucket = pricing_tools.create_vega_bucket(dop_pricing_data,bucket_strikes, bucket_expiries, shift, 
                                               scale_x=setting[parametrization][0], scale_t=setting[parametrization][1], 
                                               bucket_type =parametrization )

In [ ]:
x, y = meshgrid(bucket_strikes, bucket_expiries)
rcParams['figure.figsize'] = 6,4
plt.pcolor(x,y,vega_bucket)
plt.colorbar()
plt.xlabel('strike')
plt.xticks(bucket_strikes)
plt.ylabel('ttm')
#print(vega_bucket)

In [ ]:
dop_pricing_data.pricingRequest.setVega(True)
vega_scale = 0.01
dop_pricing_data.pricingRequest.setVegaScale(vega_scale)
vegas = analytics.price(dop_pricing_data).getVegas()

In [ ]:
vega_from_buckets = vega_scale * np.sum(vega_bucket)
vega = vegas['TEST_SURFACE']
print('vega: ' + str(vega) + ', sum of buckets: ' + str(vega_from_buckets))

### Parameter Study

In [ ]:
def create_bucket_surface(bucket_type, bucket_scale_x, bucket_scale_t, shift):
    vol_bucket_surface = analytics.VolatilitySurfaceBucketShifted(vol_surf, bucket_strikes, bucket_expiries, shift)
    # coment/uncomment to use the different bucket parametrizations
    if bucket_type == 'RBF':
        vol_bucket_surface.setRBF(bucket_scale_x, bucket_scale_t)
    else:
        vol_bucket_surface.setSimpleExponentialLocalSupport(bucket_scale_x, bucket_scale_t)
        
t = 'RBF'
result = []
if True:
    for bucket_scale_x in [1, 10]: # , 1000, 2000]:
        for bucket_scale_t in [1, 10]: #, 1000, 2000]:
            for shift in [0.005, 0.002, 0.001, 0.0005]:
                vega_bucket = pricing_tools.create_vega_bucket(dop_pricing_data,bucket_strikes, bucket_expiries, shift, scale_x=bucket_scale_x, scale_t=bucket_scale_t, bucket_type = t)
                vega_from_buckets = vega_scale * np.sum(vega_bucket)
                result.append( {'type': t, 'bucket_scale_x': bucket_scale_x, 'bucket_scale_t' : bucket_scale_t, 'shift': shift, 'vega_bucket': vega_from_buckets, 'vega': vega, 'abs(diff)': abs(vega_from_buckets - vega)} )
            

t = 'LOCAL_RBF'
for bucket_scale_x in [100, 1000]: # , 1000, 2000]:
    for bucket_scale_t in [100, 1000]: #, 1000, 2000]:
        for shift in [0.005, 0.002]: #, 0.001, 0.0005]:
            vega_bucket = pricing_tools.create_vega_bucket(dop_pricing_data,bucket_strikes, bucket_expiries, shift, scale_x=bucket_scale_x, scale_t=bucket_scale_t, bucket_type = t)
            vega_from_buckets = vega_scale * np.sum(vega_bucket)
            result.append( {'type': t, 'bucket_scale_x': bucket_scale_x, 'bucket_scale_t' : bucket_scale_t, 'shift': shift, 'vega_bucket': vega_from_buckets, 'vega': vega, 'abs(diff)': abs(vega_from_buckets - vega)} )
pd.DataFrame(result)  

### Vega hedge

In [ ]:
#analytics.setLogLevel('DEBUG')
bucket_scale_x = 500.0 #1.0 #1000
bucket_scale_t = 250.0 #1.0 #500
bucket_type = 'RBF' #'LOCAL_RBF' #'RBF'
vega_hedge = pricing_tools.compute_vega_hedge(dop_pricing_data,bucket_strikes, bucket_expiries, shift, scale_x=bucket_scale_x, 
                                              scale_t=bucket_scale_t, bucket_type = bucket_type,  optimMethod='Powell', 
                       optimOptions = {'xtol': 1e-5, 'ftol': 1e-6, 'maxiter': 100, 'maxfev': 20000, 'disp': True})

In [ ]:
#print(vega_hedge)
#plt.style.use('default')
x, y = meshgrid(bucket_strikes, bucket_expiries)
rcParams['figure.figsize'] = 6,4
plt.pcolor(x,y, vega_hedge)
plt.colorbar()
plt.xlabel('strike')
plt.xticks(bucket_strikes)
plt.ylabel('ttm')   

### General hedging

In [ ]:
def setup_ssvi_vol_scenarios(base_vol):
    scenarios = []
    ssvi = analytics.VolatilityParametrizationSSVI.fromVolParametrization(base_vol.getVolParametrization())
    expiries = ssvi.getExpiryTimes()
    atm_vols =  ssvi.getAtmfVols()
    gamma = ssvi.getGamma()
    rho = ssvi.getRho()    
    eta = ssvi.getEta()
    refdate = analytics.ptime()
    base_vol.getRefDate(refdate)
    ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, atm_vols, rho, 1.1 * eta, gamma)
    vol_surf_eta_up = analytics.VolatilitySurface('ETA_UP', refdate, base_vol.getForwardCurve(), 
                                           enums.DayCounter.ACT365_FIXED, ssvi_param)
    scenarios.append(vol_surf_eta_up)
    
    ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, atm_vols, 1.1*rho, eta, gamma)
    vol_surf_rho_up = analytics.VolatilitySurface('ETA_UP', refdate, base_vol.getForwardCurve(), 
                                           enums.DayCounter.ACT365_FIXED, ssvi_param)
    scenarios.append(vol_surf_rho_up)
    
    ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, atm_vols, rho, 1.1*eta, gamma)
    vol_surf_eta_up = analytics.VolatilitySurface('ETA_UP', refdate, base_vol.getForwardCurve(), 
                                           enums.DayCounter.ACT365_FIXED, ssvi_param)
    scenarios.append(vol_surf_eta_up)
    
    ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, atm_vols, rho, eta, 1.1* gamma)
    vol_surf_gamma_up = analytics.VolatilitySurface('ETA_UP', refdate, base_vol.getForwardCurve(), 
                                           enums.DayCounter.ACT365_FIXED, ssvi_param)
    scenarios.append(vol_surf_gamma_up)
    
    
    base_vol.getRefDate(refdate)
    for i in range(len(atm_vols)):
        shifted_atm_vol = analytics.vectorDouble([x for x in atm_vols])
        shifted_atm_vol[i] += 0.005
        ssvi_param = analytics.VolatilityParametrizationSSVI(expiries, shifted_atm_vol, rho, eta, gamma)
        vol_surf_atm = analytics.VolatilitySurface('ETA_UP', refdate, base_vol.getForwardCurve(), 
                                               enums.DayCounter.ACT365_FIXED, ssvi_param)
        scenarios.append(vol_surf_atm)
    return scenarios
    
    
hedge_ins = [{'PAYOFF': 'P', 'STRIKE': 70.0, 'EXPIRY': analytics.ptime(2017,6,1,0,0,0)},
             {'PAYOFF': 'P', 'STRIKE': 75.0, 'EXPIRY': analytics.ptime(2017,6,1,0,0,0)},
            {'PAYOFF': 'C', 'STRIKE': 100.0, 'EXPIRY': analytics.ptime(2017,6,1,0,0,0)},
            {'PAYOFF': 'C', 'STRIKE': 110.0, 'EXPIRY': analytics.ptime(2017,6,1,0,0,0)},
            {'PAYOFF': 'P', 'STRIKE': 70.0, 'EXPIRY': analytics.ptime(2018,1,1,0,0,0)},
            {'PAYOFF': 'P', 'STRIKE': 75.0, 'EXPIRY': analytics.ptime(2018,1,1,0,0,0)},
            {'PAYOFF': 'C', 'STRIKE': 100.0, 'EXPIRY': analytics.ptime(2018,1,1,0,0,0)},
            {'PAYOFF': 'C', 'STRIKE': 110.0, 'EXPIRY': analytics.ptime(2018,1,1,0,0,0)}]    

scenarios = setup_ssvi_vol_scenarios(vol_surf)
scenario_weights = [1.0 for x in scenarios]

In [ ]:
pricing_tools.compute_hedge(dop_pricing_data, scenarios, scenario_weights, hedge_ins,  optimOptions = {'xtol': 1e-3, 'ftol': 1e-3, 'maxiter': 20, 'maxfev': 1000, 'disp': True})